In [1]:
import torch
from torchinfo import summary
import s3prl.hub as hub

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
model = getattr(hub, "tera")()  # build the TERA model with pre-trained weights
print(model)

Using cache found in /home/chihyuan/.cache/torch/hub/s3prl_cache/96e0c8360819d808c1a7046af5dd8a8c62dc30fce5011342c1468f34029f7cbb
for https://www.dropbox.com/s/xdoj9wdo87lztv1/states-1000000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config
UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer

In [3]:
summary(model)

Layer (type:depth-idx)                                                 Param #
UpstreamExpert                                                         --
├─PretrainedTransformer: 1-1                                           --
│    └─OnlinePreprocessor: 2-1                                         --
│    │    └─MelScale: 3-1                                              --
│    │    └─MFCC: 3-2                                                  --
│    └─TransformerModel: 2-2                                           --
│    │    └─ModuleDict: 3-3                                            --
│    │    └─ModuleDict: 3-4                                            --
│    │    └─TransformerInputRepresentations: 3-5                       63,744
│    │    └─TransformerEncoder: 3-6                                    21,263,616
│    │    └─PrefixTuningPool: 3-7                                      --
Total params: 21,327,360
Trainable params: 21,327,360
Non-trainable params: 0

In [4]:
type(model.transformer.model)  # the model with adpaters' functions

s3prl.upstream.mockingjay.model.TransformerModel

In [5]:
model.transformer.model.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [6]:
model.transformer.model.add_adapter("acu", "lora")
print(model)

UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerLayer(
            (attention): TransformerAttention(
              (self): TransformerSelfAttention(
                (query): Linear(
                  in_features=768, out_features=768, bias=True
                  (lor

In [7]:
model.transformer.model.train_adapter("acu")
summary(model)

Layer (type:depth-idx)                                                 Param #
UpstreamExpert                                                         --
├─PretrainedTransformer: 1-1                                           --
│    └─OnlinePreprocessor: 2-1                                         --
│    │    └─MelScale: 3-1                                              --
│    │    └─MFCC: 3-2                                                  --
│    └─TransformerModel: 2-2                                           --
│    │    └─ModuleDict: 3-3                                            --
│    │    └─ModuleDict: 3-4                                            --
│    │    └─TransformerInputRepresentations: 3-5                       (63,744)
│    │    └─TransformerEncoder: 3-6                                    21,337,344
│    │    └─PrefixTuningPool: 3-7                                      --
Total params: 17,857,536
Trainable params: 73,728
Non-trainable params: 17,783,808

In [8]:
summary(model, depth=20, row_settings=["depth", "var_names"])

Layer (type (var_name):depth-idx)                                                     Param #
UpstreamExpert                                                                        --
├─PretrainedTransformer (transformer): 1-1                                            --
│    └─OnlinePreprocessor (extracter): 2-1                                            --
│    │    └─MelScale (_melscale): 3-1                                                 --
│    │    └─MFCC (_mfcc_trans): 3-2                                                   --
│    │    │    └─AmplitudeToDB (amplitude_to_DB): 4-1                                 --
│    │    │    └─MelSpectrogram (MelSpectrogram): 4-2                                 --
│    │    │    │    └─Spectrogram (spectrogram): 5-1                                  --
│    │    │    │    └─MelScale (mel_scale): 5-2                                       --
│    └─TransformerModel (model): 2-2                                                  --
│    │    └─Modu

### Psuedo Input

In [7]:
model.transformer.model.active_adapters

Stack[acu]

In [10]:
# model.transformer.model.add_adapter("acu", "pfeiffer")  # Before activated, resul is the same as original mockingjay in original s3prl
wavs = [torch.ones(160000, dtype=torch.float) for _ in range(16)]
model.train()  # will have random effect
# model.eval()
with torch.no_grad():
    print(model(wavs))

{'last_hidden_state': tensor([[[-0.0772,  0.1864, -0.1726,  ...,  0.0657, -0.4511,  0.6702],
         [ 0.0217,  0.1596, -0.1643,  ..., -0.3044, -0.2634,  0.7669],
         [ 0.0647,  0.2621, -0.2537,  ..., -0.2939, -0.2514,  0.4612],
         ...,
         [-0.0192,  0.0971, -0.1113,  ..., -0.3518, -0.3453,  0.6550],
         [ 0.0911,  0.2704, -0.0469,  ..., -0.4236, -0.2951,  0.7091],
         [-0.0363,  0.2406, -0.2056,  ..., -0.2235, -0.3272,  0.6734]],

        [[-0.0332,  0.2785, -0.0066,  ..., -0.3213, -0.3547,  0.7663],
         [-0.0694,  0.1105, -0.1702,  ..., -0.3618, -0.3082,  0.6899],
         [ 0.1156,  0.1406, -0.0776,  ..., -0.3099, -0.3621,  0.2525],
         ...,
         [-0.0385,  0.0461, -0.0262,  ..., -0.3229, -0.2217,  0.6997],
         [-0.1039,  0.1706, -0.0059,  ..., -0.3928, -0.2670,  0.7557],
         [-0.0702,  0.2806, -0.0604,  ..., -0.3823, -0.2598,  0.7267]],

        [[ 0.0136,  0.1547, -0.2302,  ..., -0.1992, -0.3561,  0.7864],
         [-0.0333,  0.0

In [8]:
model.transformer.model.delete_adapter("acu")
# model.transformer.model.add_adapter("acu", "pfeiffer")  # Before activated, resul is the same as original mockingjay in original s3prl
wavs = [torch.ones(160000, dtype=torch.float) for _ in range(16)]
# model.train()  # will have random effect
model.eval()
with torch.no_grad():
    print(model(wavs))

{'last_hidden_state': tensor([[[-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         ...,
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101]],

        [[-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         ...,
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101]],

        [[-0.0130,  0.0918, -0.1565,  ..., -0.3166, -0.3251,  0.6101],
         [-0.0130,  0.0